In [ ]:
from extractors.KeyExtract import KeyExtractor
from evaluation.evaluation import evaluate_concepts, read_json_from_file

json_data = read_json_from_file("NER_prerequisites.json")
extractor = KeyExtractor()
evaluation_data = []
for item in json_data:
    prerequisite_text = item.get('prerequisite_text', '')
    gt_concepts = item.get('concepts', [])
    extracted_concepts = extractor.extractKeyWords(prerequisite_text)
    evaluation_data.append((gt_concepts, extracted_concepts))
evaluate_concepts("", evaluation_data, threshold=0.99)

Precision: 0.74
Recall: 0.72
F1 Score: 0.72
Precision@O: 0.76
Recall@O: 0.71
F1@O Score: 0.73


In [4]:
from extractors.extractors import extractors
from evaluation.evaluation import evaluate_concepts, read_json_from_file

json_data = read_json_from_file("NER_prerequisites.json")
for name, extractor in extractors.items():
    evaluation_data = []
    for item in json_data:
        prerequisite_text = item.get('prerequisite_text', '')
        gt_concepts = item.get('concepts', [])
        extracted_concepts = extractor.extractKeyWords(prerequisite_text, int(len(gt_concepts)*1.5))
        evaluation_data.append((gt_concepts, extracted_concepts))
    evaluate_concepts(name, evaluation_data, threshold=0.99)

### Position Rank
Precision: 0.52
Recall: 0.68
F1 Score: 0.59
Precision@O: 0.55
Recall@O: 0.55
F1@O Score: 0.55


### YAKE
Precision: 0.47
Recall: 0.64
F1 Score: 0.54
Precision@O: 0.54
Recall@O: 0.54
F1@O Score: 0.54


### Key BERT
Precision: 0.39
Recall: 0.51
F1 Score: 0.44
Precision@O: 0.40
Recall@O: 0.40
F1@O Score: 0.40


### RAKE
Precision: 0.44
Recall: 0.58
F1 Score: 0.49
Precision@O: 0.46
Recall@O: 0.46
F1@O Score: 0.46


### TextRank
Precision: 0.53
Recall: 0.70
F1 Score: 0.60
Precision@O: 0.56
Recall@O: 0.55
F1@O Score: 0.56


